In [6]:
import os
import requests
import base64
from io import BytesIO

from PIL import Image
import openai

def encode_image(image):
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")
    
    
def get_current_synoptic_analysis(maps, map_urls):
    api_key = os.getenv("OPENAI_API_KEY")
    client = openai.OpenAI(api_key=api_key)
    prompt = f"""
        Please analyze these weather maps and provide a synoptic breakdown.
        Your audience is a meteorologist, so please be technical.
        Please provide your response in markdown. The map urls are {", ".join(map_urls)}.
        Please include the urls in the markdown.
    """
    system = "You are a helpful meteorological assistant."

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": [{"type": "text", "text": prompt}]}
    ]

    for image in maps:
        base64_img = encode_image(image)
        messages[1]["content"].append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_img}"
            }
        })

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content


def download_files(date, hour):
    map_urls = [
        "https://climate.cod.edu/data/surface/US/contour/current/US.fronts.gif", # sfc
        f"https://climate.cod.edu/data/upper/US/contour/USvort.{date}.{hour}.gif", # 500mb vort
        f"https://climate.cod.edu/data/upper/US/700/US700rh.{date}.{hour}.gif", # 700mb rh
        f"https://climate.cod.edu/data/upper/US/850/US850dew.{date}.{hour}.gif", # 850 dpt
        f"https://climate.cod.edu/data/upper/US/contour/USthick.{date}.{hour}.gif" #1000-500mb del Z
    ]
    maps = []
    for url in map_urls:
        response = requests.get(url)
        image = Image.open(BytesIO(response.content))
        maps.append(image)

    return maps, map_urls

In [10]:
from datetime import datetime
date = datetime.now().strftime("%Y%m%d")
hour = 12
maps, map_urls = download_files(date, hour)
answer = get_current_synoptic_analysis(maps, map_urls)

In [12]:
from IPython.display import Markdown, display
display(Markdown(answer))

## Synoptic Weather Analysis

### Surface Analysis
![Surface Map](https://climate.cod.edu/data/surface/US/contour/current/US.fronts.gif)

A well-defined low-pressure system is positioned over the northwestern United States, with an associated cold front extending southeastward. There’s another smaller low-pressure system over the southeastern United States, with a warm front visible extending northeast. High-pressure areas are stationed over the central plains and northeastern states, creating a pressure gradient that is driving northeasterly winds in these regions.

### 500 mb Vorticity
![500 mb Vorticity](https://climate.cod.edu/data/upper/US/contour/USvort.20250511.12.gif)

The 500 mb chart shows a deep trough over the southeastern United States, indicating potential cyclonic activity and possibly enhanced vertical motion. High vorticity is concentrated around the Gulf of Mexico, suggesting active weather and convective potential. Upper-level convergence over the Pacific Northwest aligns with the surface high-pressure area.

### 700 mb Relative Humidity
![700 mb RH](https://climate.cod.edu/data/upper/US/700/US700rh.20250511.12.gif)

The 700 mb relative humidity map shows increased moisture over the southeastern United States and Gulf Coast, correlating with the trough and surface low observed earlier. Dry air is predominant over the central plains, suggesting a ridge or subsidence at mid-levels.

### 850 mb Dew Point
![850 mb Dew Point](https://climate.cod.edu/data/upper/US/850/US850dew.20250511.12.gif)

The 850 mb map highlights a moist air mass throughout the southeastern United States, with higher dew points reinforcing the potential for convection and precipitation. There is also a noticeable moisture gradient from east to west, supporting frontal activity and possible severe weather conditions.

### 1000-500 mb Thickness 
![Thickness](https://climate.cod.edu/data/upper/US/contour/USthick.20250511.12.gif)

The thickness map shows a significant thermal gradient across the northeastern and midwestern United States, indicating a baroclinic zone that is often associated with active frontal boundaries. The tight packing of isobars and thickness lines in the southwest suggests strong winds and possible jet streak dynamics.

### Summary

The current synoptic setup shows a complex system with significant low-pressure activity in both northwest and southeast United States. The southeast is particularly primed for convective weather due to the combination of upper-level vorticity, high moisture content, and lifting mechanisms. Meanwhile, high pressure continues to dominate the central United States, providing more stable but cooler conditions.

This analysis should provide a comprehensive understanding of the current weather patterns influencing the United States. The presented data supports the potential for active weather in regions aligned with current low-pressure systems and associated upper dynamics.